# `unittest.mock`
**Replacing objects to test how they're used**

[`unittest.mock` main documentation](https://docs.python.org/3/library/unittest.mock.html)
[Getting started](https://docs.python.org/3/library/unittest.mock-examples.html

* Basic idea "It allows you to replace parts of your system under test with mock objects and make assertions about how they have been used."
* Mock class / MagicMock class 
                  "MagicMock is a subclass of Mock with default implementations of most of the magic methods."
    * Create attributes as they are accessed
    * Specify return values, limitations, etc.
    * record details of how they're used
                  * called, call_count, call-args, call_args_list, method_calls, mock-calls
    * Assert about how they're used
                  * various assertions
    * side_effect: TODO
                  * callable
                  * list
                  * exception class or instance
    * return value
    * name
    * spec: TODO
    * reset ,add_spec, configure_mock,
    * non-callable versions
    * child-parent relationship / mock as attributes of mocks
                  * naming vs. not naming mock attributes
    
* patch():
                  * class decorator
                  * function decorator
                  * context manager
                  * patching `TestCase` subclasses
* patch.object
* patch.dict: TODO
* autospeccing, create_autospec
* patch() and Mock() take arbitrary keywords
* mock_open

# `unittest.mock.Mock`
*Flexibly replace objects for testing purposes*

* Creates attributes on access
* Simulates callable and non-callable objects
* Records how it is used

In [56]:
from unittest.mock import Mock

# accessing mocks creates new mocks by default
m = Mock()
m
# m()
# m.foo
# m.foo()

<Mock name='mock.foo()' id='4573556920'>

## Return values for `Mock`s

You can specify a return value in two ways:

1. The `return_value` initializer argument
2. The `return_value` attribute

In [57]:
# Set the return value via the constructor
m = Mock(return_value=42)
m()

42

In [58]:
# Or via the `return_value` attribute
m.return_value = "'I don't know what I was expecting.' -- Michael Bloom"
m()

"'I don't know what I was expecting.' -- Michael Bloom"

## Side-effects for `Mocks`
*An object determing how a `Mock` behaves when called*

Specify a side-effect in two ways:
1. The `side_effect` initializer argument
2. The `side_effect` attribute

In [59]:
m = Mock(side_effect=lambda x: x * 2)
m.side_effect = [1,2,3]

### Callable side-effects
*Return value of callable is return value of calling the mock*

In [ ]:
# Callable side-effects produce the Mock's return value
def add(x, y):
    return x + y

m = Mock(side_effect=add)
m(1, 2)
# m("ambi", "dextrous")

In [ ]:
# The side-effect has normal function arity, etc.
m(1,2,3)

### Exception side-effects
*Exception instances and classes cause a throw when the mock is called*

In [62]:
# An exception type creates a new instance for each call
m = Mock(side_effect=ValueError)
try:
    m()
except ValueError:
    print('exception raised!')

exception raised!


In [66]:
# An exception instance uses the same instance for each call
m.side_effect = ValueError("oops!")
try:
    m()
except ValueError as e:
    assert e.args[0] == "oops!"

### Iterable side-effects
*The iterable's values are the consecutive return values of the mock*

In [67]:
m = Mock(side_effect=[1,2,3])
m(), m(), m()

(1, 2, 3)

##  `Mock` remembers how it is used
*This is the core of how `Mock` can be tested*

* Determine if and/or how many times it was called
* See the arguments used when it is called

In [74]:
m = Mock()
m()
"Called? {}. Call count = {}.".format(
    m.called,
    m.call_count)

'Called? True. Call count = 1.'

###  `call_args` and `call_args_list` track the arguments to `Mock` calls

In [79]:
# `call_args` get the *last* argument set. `call_args_list` gets all of them.
m = Mock()
m(1,2,3)
m("This", "that", last="the other")
m.call_args
# m.call_args_list

call('This', 'that', last='the other')

### `method_calls` tracks calls to methods (and methods on methods!) on the `Mock`

In [85]:
m = Mock()
m() # This is *not* reported in `method_calls`
m.some_method()
m.what.about.demeter("?!")
m.method_calls

[call.some_method(), call.what.about.demeter('?!')]

### `mock_calls` tracks everything

In [89]:
m = Mock()
# `mock_calls` tracks...
m(1337)              # ...calls to the Mock
m.attr("")           # ...calls to attributes of the Mock
m.deep(1).linking(2) # ...Call to the results of calls to attributes 
m.mock_calls

[call(1337), call.attr(''), call.deep(1), call.deep().linking(2)]